<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/CA5/mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install sacremoses
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=b504bd7450d135801b133045cb0271db8929ad93352d0874ef26f3067d8d2a54
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [3]:
import os
import json
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch

In [4]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:  
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
BASE_PATH = "/content/drive/MyDrive/NLP/CA5"
TOKENIZED_DATA_PATH = os.path.join(BASE_PATH, "mbert_tokenized_data")
TRAIN_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "train")
VALID_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "valid")
TEST_DATA_PATH = os.path.join(TOKENIZED_DATA_PATH, "test")
DATA_DIR_PATH = os.path.join(BASE_PATH, "mbert_data_dir")

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = "This is a test"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
!mkdir -p ./data_bin

In [8]:
!fairseq-preprocess   --joined-dictionary	 --source-lang en --target-lang fa \
  --trainpref {TRAIN_DATA_PATH} \
  --validpref {VALID_DATA_PATH} \
  --testpref {TEST_DATA_PATH} \
  --destdir {DATA_DIR_PATH}

2023-06-10 16:03:30.763421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-10 16:03:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-06-10 16:03:34 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

In [89]:
with open(os.path.join(DATA_DIR_PATH, "dict.en.txt"), "r") as f:
    mbert_dict = f.read().splitlines()

In [79]:
len(mbert_dict)

33212

In [90]:
mbert_dict_normalized = {}
for index,item in enumerate(mbert_dict):
    if item.split()[1].isdigit():
        mbert_dict_normalized[item.split()[0]] = item.split()[1]
    else:
        mbert_dict_normalized[item.split()[1]] = item.split()[0]

In [93]:
len(mbert_dict_normalized)

33212

In [138]:
import random
subset = dict(random.sample(mbert_dict_normalized.items(),100))
chosen_set = mbert_dict_normalized

<ipython-input-138-7ea466ab00f6>:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset = dict(random.sample(mbert_dict_normalized.items(),100))


In [139]:
chosen_set

{'[CLS]': '1340000',
 '[SEP]': '1340000',
 '.': '1181288',
 ',': '1173223',
 'the': '723684',
 'در': '566759',
 'و': '481315',
 '##ی': '441556',
 'به': '411359',
 '"': '394985',
 'of': '386050',
 'که': '368892',
 'to': '339876',
 'از': '331705',
 'and': '325850',
 'in': '296914',
 '-': '294690',
 'م': '286932',
 'را': '277547',
 '##ه': '269652',
 'این': '242466',
 'است': '231154',
 'a': '218869',
 '##s': '177295',
 'ا': '173475',
 'ن': '170210',
 'با': '160739',
 '##د': '158762',
 '##م': '149064',
 "'": '147974',
 '##ان': '142768',
 'that': '137017',
 'برای': '120469',
 '##ت': '118206',
 'ت': '117536',
 'یک': '117009',
 'for': '112609',
 'is': '108521',
 'بر': '107076',
 'خ': '103833',
 'The': '101506',
 's': '100513',
 '##ات': '99759',
 'تا': '99546',
 '##ا': '99187',
 '##های': '96139',
 'می': '94692',
 'های': '94266',
 '##ن': '93701',
 'said': '93088',
 ':': '92286',
 'گفت': '90696',
 'ب': '89006',
 '(': '88290',
 ')': '84309',
 'ق': '84080',
 '##ش': '83277',
 'او': '82989',
 'on': '

In [ ]:
mbert_dict_embeddings = {}

for key in tqdm(chosen_set.keys()):
    encoded = tokenizer.encode(key)
    input_ids = torch.tensor([encoded])

    with torch.no_grad():
      embedding = model(input_ids)[0].mean(1)[0].tolist()

    mbert_dict_embeddings[key] = embedding



  1%|          | 390/33212 [00:26<31:03, 17.61it/s]

In [ ]:
final_list = []

for key,value in mbert_dict_embeddings.items():
  embd = ' '.join([str(x) for x in value])
  sample = f"{key} {embd}"
  final_list.append(sample)

In [ ]:
with open(os.path.join(".","mbert_dict_embeddings.txt"), 'w') as f:
    f.write(str(len(chosen_set)))
    f.write(" ")
    f.write(str(len(list(chosen_set.values())[0])))
    f.write("\n")
    f.write("\n".join(final_list))


In [ ]:
with open('./mbert_dict_embeddings.txt',"r") as f:
  dummy = f.readlines()

In [ ]:
dummy[4]

In [ ]:
!fairseq-train \
  /content/drive/MyDrive/NLP/CA5/mbert_data_dir \
  --arch lstm --share-decoder-input-output-embed --share-all-embeddings --encoder-embed-dim 768 --decoder-embed-dim 768 --decoder-out-embed-dim 768 --encoder-embed-path /content/mbert_dict_embeddings.txt   \
  --optimizer adam --adam-betas '(0.9,0.98)' --clip-norm 0.0 \
  --lr 2.5e-3 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
  --dropout 0.25 --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --max-tokens 4096 \
  --eval-bleu \
  --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
  --eval-bleu-detok moses \
  --eval-bleu-print-samples \
  --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
  --fp16 --memory-efficient-fp16 \
  --max-epoch 5 \
  --save-dir ./content/drive/MyDrive/mbert/checkpoints/ \
  --tensorboard-logdir ./content/drive/MyDrive/mbert/log/



In [ ]:
!ls /content/drive/MyDrive/NLP/CA5/mbert_tokenized_data